In [44]:
import os
import sys
import matplotlib.pyplot as plt
import numpy as np
import pydicom as di
import pandas as pd
import tensorflow as tf
from PIL import Image
import pydicom as pydcm
import cv2

In [52]:
# Loading the CSV file and cleaning the data
train_csv = pd.read_csv("./train.csv")
train_csv.head(10)

,site_id,patient_id,image_id,laterality,view,age,cancer,biopsy,invasive,BIRADS,implant,density,machine_id,difficult_negative_case
0,2,10006,462822612,L,CC,61.0,0,0,0,NaN,0,NaN,29,False
1,2,10006,1459541791,L,MLO,61.0,0,0,0,NaN,0,NaN,29,False
2,2,10006,1864590858,R,MLO,61.0,0,0,0,NaN,0,NaN,29,False
3,2,10006,1874946579,R,CC,61.0,0,0,0,NaN,0,NaN,29,False
4,2,10011,220375232,L,CC,55.0,0,0,0,0.0,0,NaN,21,True
5,2,10011,270344397,L,MLO,55.0,0,0,0,0.0,0,NaN,21,True
6,2,10011,541722628,R,CC,55.0,0,0,0,NaN,0,NaN,21,False
7,2,10011,1031443799,R,MLO,55.0,0,0,0,NaN,0,NaN,21,False
8,2,10025,562340703,L,CC,75.0,0,0,0,NaN,0,NaN,29,False
9,2,10025,1365269360,L,MLO,75.0,0,0,0,NaN,0,NaN,29,False


In [14]:
#Exploring data
print(f"<-- Difficult negative case --> \n {train_csv['difficult_negative_case'].value_counts()}\n")

print(f"<-- Cancer --> \n {train_csv['cancer'].value_counts()}\n")

# The data is very skewed.

<-- Difficult negative case --> 
 False    47001
True      7705
Name: difficult_negative_case, dtype: int64

<-- Cancer --> 
 0    53548
1     1158
Name: cancer, dtype: int64



In [30]:
# Getting a csv of the files we have.
patient_ids_on_hand = []
#Getting the names of the folders at hand.
for file in os.listdir("./images"):
    patient_ids_on_hand.append(int(file))
len(patient_ids_on_hand)

200

In [38]:
# Creating a CSV of images on hand.
csv_data_on_hand = pd.DataFrame(columns=train_csv.columns)

for patient_id in patient_ids_on_hand:
   csv_data_on_hand =  pd.concat([csv_data_on_hand, train_csv[train_csv['patient_id'] == patient_id]])


In [43]:
# Looking at distribution of postivie values.
csv_data_on_hand["cancer"].value_counts()

0    874
1     18
Name: cancer, dtype: int64

In [50]:
#Fucntion to convert dcm images to nparray of type uint16
def dcm_to_numpy(path_to_dcmimage):
    dcm_image = di.dcmread(path_to_dcmimage)
    np_array = dcm_image.pixel_array
    return np_array

def save_dcm_as_png(path_to_dcm, png_save_path):
    im = Image.fromarray(dcm_to_numpy(path_to_dcm)*128)
    im.save(png_save_path)